# **NYC Trip Fare Analysis**

In [1]:
import sys
import os

# Aggiungi la cartella 'conf' a sys.path (se non già presente)
conf_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'conf'))
if conf_path not in sys.path:
    sys.path.append(conf_path)

# Ora puoi importare il modulo di configurazione
import conf

# Verifica che i percorsi siano corretti
print("Main directory:", conf.MAIN_DIR)
print("Notebook directory:", conf.NOTEBOOK_DIR)
print("Data directory:", conf.DATA_DIR)

Main directory: /Users/sara/Desktop/Magistrale - Data Science/Computer Science/Project/foc-project-nava-perani
Notebook directory: /Users/sara/Desktop/Magistrale - Data Science/Computer Science/Project/foc-project-nava-perani/notebooks
Data directory: /Users/sara/Desktop/Magistrale - Data Science/Computer Science/Project/foc-project-nava-perani/data


In [2]:
import pandas as pd

In [4]:
dataset = pd.read_csv(f'{conf.DATA_DIR}/data.csv')

/var/folders/w7/0xpq0vds7cz2333831nfq5dc0000gn/T/ipykernel_41255/3836098578.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f'{conf.DATA_DIR}/data.csv')


In [5]:
dataset.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0


In [7]:
print(dataset.dtypes)

VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object
